In [ ]:
# Install necessary libraries 

!pip install -q --upgrade keras-cv
!pip install -q --upgrade keras  # Upgrade to Keras 3.
!pip install gradio



In [ ]:
# Import required libraries

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
import keras_cv




In [ ]:
# Mount Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define class names for animal classification

class_names = ['Bear', 'Bird', 'Cat', 'Cow', 'Deer', 'Dog', 'Dolphin', 'Elephant', 'Giraffe', 'Horse', 'Kangaroo', 'Lion', 'Panda', 'Tiger', 'Zebra']

In [ ]:
# Load a pre-trained ResNet50V2 model from KerasCV

model = keras_cv.models.ImageClassifier.from_preset(
    "resnet50_v2_imagenet", num_classes=len(class_names)
)

In [ ]:
# Import optimization and callback modules for training

from tensorflow.keras.optimizers import RMSprop, Adam, AdamW
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint

# Define a checkpoint to save the best model weights during training.

checkpoint_filepath = 'model/weights-improvement-{epoch:02}-{val_loss:.4f}.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2, monitor='val_acc', mode='max',
    save_best_only=True
)

In [ ]:
# Define early stopping to prevent overfitting.

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=15,
                        verbose=2, restore_best_weights=True)

# Compile the model with AdamW optimizer, sparse categorical crossentropy loss, and accuracy metric.
model.compile(optimizer=AdamW(learning_rate=0.001),loss='sparse_categorical_crossentropy',
              metrics = ['acc'])



In [ ]:
# Load pre-trained weights 

model.load_weights("model weights location ")

In [ ]:
# Import PIL for image processing

from PIL import Image
import numpy as np

# Load and preprocess a sample image for testing.

path = " test image path " 
image = Image.open(path)
image = np.array(image)
image = image / 255.0  
image = np.expand_dims(image, axis=0)  


In [ ]:
# Resize the image using KerasCV resizing layer.

resizing = keras_cv.layers.Resizing(
    224, 224, crop_to_aspect_ratio=True
)
np_im_rs = resizing(image)
print(np_im_rs.shape)


In [ ]:
# Make predictions on the preprocessed image.

predictions = model.predict(np_im_rs)
print(predictions)

# Print the predicted class.

print("Top class is:", class_names[np.argmax(predictions[0])])


In [ ]:
# Define a function to classify an image from a file path.
def classify_image(path):
    image = Image.open(path)
    image = np.array(image)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)

    resizing = keras_cv.layers.Resizing(
        224, 224, crop_to_aspect_ratio=True
    )
    np_im_rs = resizing(image)

    predictions = model.predict(np_im_rs)
    return class_names[np.argmax(predictions[0])]


In [ ]:
# Create a Gradio interface for image classification.

gr.Interface(fn=classify_image,
             inputs=gr.Image(type="filepath"), # Input is an image file path.
             outputs="text").launch(debug='True') # Output is text (predicted class)